In [13]:
import openai
OPENROUTER_API_KEY = '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'
YOUR_SITE_URL = 'www'
YOUR_APP_NAME = 'demo'
MODEL = 'openai/gpt-3.5-turbo-instruct'

# google/gemma-7b-it
# mistralai/mixtral-8x7b-instruct
# meta-llama/llama-2-70b-chat
# openai/gpt-3.5-turbo-instruct

# Define your constraints
max_tokens = 500  # Set your desired maximum word count here

# OpenRouter API base URL
openai.api_base = "https://openrouter.ai/api/v1"
openai.api_key = OPENROUTER_API_KEY


In [14]:
from os import getenv
from openai.error import APIError
import time

def use_model(model_name, content, max_retries=5):
    for i in range(max_retries):
        try:
            completion = openai.ChatCompletion.create(
            extra_headers={
                "HTTP-Referer": YOUR_SITE_URL, # Optional, for including your app on openrouter.ai rankings.
                "X-Title": YOUR_APP_NAME, # Optional. Shows in rankings on openrouter.ai.
            },
            model= model_name,
            messages=[
                {
                "role": "user",
                "content": content
                },
            ],
            max_tokens=max_tokens

            )
            return completion.choices[0].message.content
        except APIError as e:
            print(f"An error occurred: {e}. Retrying... (Attempt {i+1}/{max_retries})")
            time.sleep(2)  # Wait for 2 seconds before retrying
    print(f"Failed after {max_retries} attempts.")
    return None

In [15]:
import pandas as pd
import time
# Load model directly

# Replace 'your_file.csv' with the actual file path
file_path_image_description = 'Jensen\BLIP-output_captions.csv'
file_path_text_description = 'Jensen\sampled_products_5_each_img_front 1.csv'

# Read the CSV file into a DataFrame
df_image = pd.read_csv(file_path_image_description)
df_text = pd.read_csv(file_path_text_description)
merged_df = pd.merge(df_text, df_image, on='ItemSku', how='left')
merged_df['Generated Product Description'] = ''
merged_df['Generated Product Features'] = ''
model_name_for_saving = MODEL.replace("/","")
model_name_for_saving = model_name_for_saving.replace(":","")

#for products with image
for index, row in merged_df.iterrows():
    if pd.isna(row["MarketingName"]):
        name = row['ProductName']
    else:
        name = row['MarketingName']
    for index, col in enumerate(["create a attractive product description in paragraphs", "create product features in bullet point"]):
        if (not pd.isna(row["MarketingDetails"])) and (not pd.isna(row["MarketingFeatures"])):
            input_text = f'Please help me {col} for this product: 1. Product name: {name} 2. Product details: {row["MarketingDetails"]} 3. Product Features: {row["MarketingFeatures"]} 4. Product Image Description: {row["UnconditionalCaption"]}'
        elif (pd.isna(row["MarketingDetails"])) and (not pd.isna(row["MarketingFeatures"])):
            input_text = f'Please help me {col} for this product: 1. Product name: {name} 2. Product Features: {row["MarketingFeatures"]} 3. Product Image Description: {row["UnconditionalCaption"]}'
        elif (not pd.isna(row["MarketingDetails"])) and (pd.isna(row["MarketingFeatures"])):
            input_text = f'Please help me {col} for this product: 1. Product name: {name} 2. Product details: {row["MarketingDetails"]} 3. Product Image Description: {row["UnconditionalCaption"]}'
        else:
            input_text = f'Please help me {col} for this product: 1. Product name: {name} 2. Product Image Description: {row["UnconditionalCaption"]}'

        output = use_model(MODEL, input_text)
        print(f"ItemSku: {row['ItemSku']}\nOutput {index}:\n{output}")

        if index == 0:
            merged_df.loc[merged_df['ItemSku'] == row['ItemSku'],['Generated Product Description']] = output
        else:
            merged_df.loc[merged_df['ItemSku'] == row['ItemSku'],['Generated Product Features']] = output

    
    merged_df.to_csv(f"Generated_text_{model_name_for_saving}.csv", index= False)
        

    



ItemSku: 19283315462
Output 0:

Sure, here is a product description for the Yankee Candle Signature Collection Small Tumbler Vanilla Cupcake:

Introducing the perfect blend of sweetness and indulgence with the Yankee Candle Signature Collection Small Tumbler Vanilla Cupcake. This 4.3 oz candle has a decadent fragrance that will fill your home with the irresistible scent of vanilla cupcakes, enriched with hints of lemon and luscious buttery icing. The fragrance notes are carefully crafted to capture the essence of this classic treat, with notes of vanilla icing, malted sugar, and cake batter at the top, followed by chocolate and cocoa in the middle, and finished off with a touch of vanilla bean.

Not only does this candle smell amazing, but it also has features that make it a must-have for any candle lover. The reimagined Tumbler shape now comes with a brushed tin lid that doubles as a coaster, making it easy to place and protect while you enjoy the delicious aroma. The cotton wick and 